In [1]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wraper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=wraper)
wiki.name

'wikipedia'

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,HuggingFaceEmbeddings())
retriever=vectordb.as_retriever()
retriever

c:\Users\Mrinal Kalita\Python Projects\RAG using langchain\myvenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020431D81B70>)

In [3]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [4]:
retriever_tool.name

'langsmith_search'

In [5]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [6]:
tools = [wiki,retriever_tool,arxiv]

In [7]:
from dotenv import load_dotenv
import os

load_dotenv()


os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 6


In [8]:
from langchain_groq import ChatGroq
llm = ChatGroq(model='llama3-8b-8192')
llm_with_tools = llm.bind_tools(tools=tools)

In [13]:
from langchain import hub
prompt = hub.pull("hwchase17/react-chat-json")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of top

In [17]:
#Agents
from langchain.agents import create_tool_calling_agent,create_structured_chat_agent, create_react_agent, create_openai_tools_agent,create_json_agent, create_json_chat_agent
agents = create_json_chat_agent(llm=llm_with_tools,tools=tools,prompt=prompt)

In [23]:
from langchain.agents import AgentExecutor

agent_exe = AgentExecutor(agent=agents, tools=tools,verbose=True,handle_parsing_errors=True)
agent_exe

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_messages(x['intermediate_steps'], template_tool_response=template_tool_response))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': [], 'to

In [24]:
agent_exe.invoke({'input':"What is langchain?"})['output']



> Entering new AgentExecutor chain...
Could not parse LLM output: Invalid or incomplete response```json
{
    "action": "Final Answer",
    "action_input": "Langchain is a platform for building and deploying large language models."
}

> Finished chain.


'Langchain is a platform for building and deploying large language models.'

# Adding Chat Hitory

In [38]:
#Managing state chat history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}

def get_message_history(session_id :str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    
    return store[session_id]

In [39]:
agent_exe_history = RunnableWithMessageHistory(agent_exe,get_message_history,input_messages_key="input",
    history_messages_key="chat_history" )

In [40]:
agent_exe_history.invoke({'input':"What is langchain?",},
                          config = {"configurable": {"session_id": "AB322"}})['output']



> Entering new AgentExecutor chain...
Could not parse LLM output: Invalid or incomplete responseCould not parse LLM output: Invalid or incomplete response```json
{
    "action": "Final Answer",
    "action_input": "LangChain is a large language model designed to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics."
}
```

> Finished chain.


'LangChain is a large language model designed to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.'

In [41]:
agent_exe_history.invoke({'input':"What is artificial intelligence?",},
                          config = {"configurable": {"session_id": "AB322"}})['output']



> Entering new AgentExecutor chain...
Could not parse LLM output: Invalid or incomplete responseCould not parse LLM output: {
    "tool_calls": [
        {
            "id": "pending",
            "type": "function",
            "function": {
                "name": "wikipedia"
            },
            "parameters": {
                "query": "Artificial intelligence"
            }
        }
    ]
}Invalid or incomplete response{
    "action": "wikipedia",
    "action_input": "Artificial intelligence"
}Page: Artificial intelligence
Summary: Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer {
    "action": "Final Answer",
    "action_input": "Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science"
}

> Finished chain.


'Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science'

In [46]:
agent_exe_history.invoke({'input':"what is data science?",},
                          config = {"configurable": {"session_id": "AB322"}})['output']



> Entering new AgentExecutor chain...
```json
{
    "action": "wikipedia",
    "action_input": "Data science"
}
```Page: Data science
Summary: Data science is an interdisciplinary academic field that uses statistics, scientific computing, scientific methods, processes, scientific visualization, algorithms and syst```json
{
    "action": "Final Answer",
    "action_input": "Data science is an interdisciplinary academic field that uses statistics, scientific computing, scientific methods, processes, scientific visualization, algorithms and systems to extract knowledge or insights from data in various forms, such as structured or unstructured data."
}

> Finished chain.


'Data science is an interdisciplinary academic field that uses statistics, scientific computing, scientific methods, processes, scientific visualization, algorithms and systems to extract knowledge or insights from data in various forms, such as structured or unstructured data.'

In [48]:
agent_exe_history.invoke({'input':"Skills needed",},
                          config = {"configurable": {"session_id": "AB322"}})['output']



> Entering new AgentExecutor chain...
Here is the response in the format of Option 1:

```json
{
    "action": "wikipedia",
    "action_input": "What are the key skills needed for a data scientist?"
}
```Page: Data science
Summary: Data science is an interdisciplinary academic field that uses statistics, scientific computing, scientific methods, processes, scientific visualization, algorithms and syst```
{
    "action": "Final Answer",
    "action_input": "Data science is an interdisciplinary academic field that uses statistics, scientific computing, scientific methods, processes, scientific visualization, algorithms and systems to extract knowledge or insights from data in various forms, such as structured or unstructured data."
}
```

> Finished chain.


'Data science is an interdisciplinary academic field that uses statistics, scientific computing, scientific methods, processes, scientific visualization, algorithms and systems to extract knowledge or insights from data in various forms, such as structured or unstructured data.'